# making an aggregate master dataframe for baseline model  10/22/18 -11/6/18
this notebook is going to make standardized longformat dataframes for each dataframe that i will adjust for each model to. 

for my first pass, i will work to establish a baseline model by using:
the single "worst", or value that most indicates poor clinical outcomes, for each variable so each variable only has one row per patient. 


- make a long format table(ie variable, patient, time, value)
 - step1:Standardize all columns, format, etc. 
 - step2: Maybe make a long format table for each dataframe
 - step3: Impute
 
- combine features from each long table for 1 wide table (ie each patient has a row, each parameter has a column). 
 - feature select for "Clinical worst case" 



- Establish a baseline (ie train model initially), using an obvious baseline: last valid mesurement of a particular variable.   Will hope that it doesn’t perform too good or too bad. Second would be to pick an aggregate within a time window (over 3 days, or of each day, ie can change graunlarity).

- Next try temporal trend, maybe vector autoregression. 

So my first step would be to pick either the last recorded value for each variable or the ‘worst’ value, or ones that we might expect to indicate poor outcome (NEED TO CHOOSE)


## last run: 
* (1) 6/9/19: sensitivity analysis 1day timewindow
* (2) 11/9/19: rerun 72day timewindow because 72hr has low patients on 

In [57]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path
import seaborn as sns
import numpy as np
import glob

from sklearn.externals.joblib import Memory
memory = Memory(cachedir='/tmp', verbose=0)
#@memory.cache above any def fxn.

%matplotlib inline
plt.style.use('ggplot')

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 15.2 ms


/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: The 'cachedir' parameter has been deprecated in version 0.12 and will be removed in version 0.14.
You provided "cachedir='/tmp'", use "location='/tmp'" instead.
  # Remove the CWD from sys.path while we load stuff.


In [58]:
#cohort import

os.chdir('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling') #use to change working directory
wd= os.getcwd() #'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling'

most_updated_patient_df= "04042019"
final_pt_df2 = pd.read_csv('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_final_pt_df2.csv'%(most_updated_patient_df), index_col=0) #only for patients with minimum vitals
patients= list(final_pt_df2['subject_id'].unique())
hadm_id= list(final_pt_df2['hadm_id'].unique())
icustay_id= list(final_pt_df2['icustay_id'].unique())
icustay_id= [int(x) for x in icustay_id]

time: 93.7 ms


In [59]:
final_pt_df2['icustay_id'].nunique() #14478

14181

time: 4.15 ms


In [60]:
#import all clinical variables
##ensure they are the versions with UOM

time: 444 µs


In [133]:
#importing in all clinical_variable files

# ##24 hr sensitivity
# #importing in all clinical_variable files
lower_window=0
upper_window=1
time_col="charttime"
time_var="t_0"
folder="24_hr_window"
timewindowdays="24"
date= '09062019'
patient_df= final_pt_df2

# #48 hr sensitivity
# lower_window=0
# upper_window=2
# time_var="t_0"
# folder="48_hr_window"
# timewindowdays="48"
# date='16052019'
# time_col="charttime"
# time_var= 't_0'
# patient_df= final_pt_df2


# # 24 hr sensitivity
# lower_window=0
# upper_window=1
# time_var="t_0"
# folder="24_hr_window"
# date='04042019'
# timewindowdays="24"

# # # 48 hr sensitivity
# lower_window=0
# upper_window=2
# time_var="t_0"
# folder="48_hr_window"
# date='16052019'
# timewindowdays="48"


# # # 78 hr elixhauser-redo 
# lower_window=0
# upper_window=3
# folder="72_hr_window"
# date='11062019'
# time_col="charttime"
# time_var= 't_0'
# timewindowdays="72"


os.chdir(r'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/processed/') #folder with all prepped clinical data stored in date_file_prepped.csv format
allFiles = glob.glob(os.getcwd() + "/{}_*.csv".format(date))

time: 12 ms


In [62]:
os.getcwd()

'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/processed'

time: 2.14 ms


In [63]:
allFiles  #need to rerun 03.1-clinical_variables and have a new date to make it easier. 

['/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/processed/16052019_leuk_prepped.csv',
 '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/processed/16052019_vasopressin_prepped.csv',
 '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/processed/16052019_bg_ART_prepped.csv',
 '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/processed/16052019_rrt_prepped.csv',
 '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/processed/16052019_epinephrine_prepped.csv',
 '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/processed/16052019_pt_info_prepped.csv',
 '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/processed/16052019_ventcategory_prepped.csv',
 '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/processed/16052019_height_prepped.csv',
 '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/processed/16052019_norepineph

time: 2.54 ms


In [64]:
#making a dictionary of all my dataframes for easier cycling through

df_list=[]
for element in allFiles:
    df_list.append(element.split('{}_'.format(date))[1].split('_prepped.csv')[0]) #making an list of all my dataframes in order they appear in file

dfs = {}
i=0
for name in df_list:
    dfs[name] = pd.read_csv(allFiles[i],  index_col=0)
    i+=1

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


time: 9.62 s


In [65]:
#all of the column names 
for element in df_list:
    print(element,':',list(dfs[element]))

leuk : ['hadm_id', 'icustay_id', 'subject_id', 't_0', 'delta', 'value', 'label', 'uom']
vasopressin : ['icustay_id', 't_0', 'value', 'label', 'uom', 'delta']
bg_ART : ['delta', 'hadm_id', 'icustay_id', 'label', 'subject_id', 't_0', 'uom', 'value']
rrt : ['icustay_id', 't_0', 'value', 'label', 'uom', 'delta']
epinephrine : ['icustay_id', 't_0', 'value', 'label', 'uom', 'delta']
pt_info : ['icustay_id', 'subject_id', 't_0', 'label', 'value', 'delta', 'uom']
ventcategory : ['icustay_id', 'delta', 'value', 'uom', 't_0', 'label']
height : ['icustay_id', 'value', 'uom', 'label', 'delta', 't_0']
norepinephrine : ['icustay_id', 't_0', 'value', 'label', 'uom', 'delta']
weight : ['icustay_id', 'value', 'uom', 'label', 'delta', 't_0']
gcs : ['subject_id', 'hadm_id', 'icustay_id', 'delta', 'label', 'value', 't_0', 'uom']
labs : ['delta', 'hadm_id', 'icustay_id', 'label', 'subject_id', 't_0', 'uom', 'value']
dobutamine : ['icustay_id', 't_0', 'value', 'label', 'uom', 'delta']
vitals : ['subject_id'

## standardizing columns

#### adding icustay_id, dropping hadm_id

In [66]:
##dropping hadm_id from all:
list1=[]
for element in df_list:
    if 'hadm_id' in (list(dfs[element])):
        list1.append(element)

for element in list1:
    dfs[element]= dfs[element].drop('hadm_id', axis=1)

time: 401 ms


In [67]:
##dropping subject_id from all:
list1=[]
for element in df_list:
    if 'subject_id' in (list(dfs[element])):
        list1.append(element)

for element in list1:
    dfs[element]= dfs[element].drop('subject_id', axis=1)

time: 363 ms


In [68]:
#all of the column names 
for element in df_list:
    print(element,':',sorted(list(dfs[element])))

leuk : ['delta', 'icustay_id', 'label', 't_0', 'uom', 'value']
vasopressin : ['delta', 'icustay_id', 'label', 't_0', 'uom', 'value']
bg_ART : ['delta', 'icustay_id', 'label', 't_0', 'uom', 'value']
rrt : ['delta', 'icustay_id', 'label', 't_0', 'uom', 'value']
epinephrine : ['delta', 'icustay_id', 'label', 't_0', 'uom', 'value']
pt_info : ['delta', 'icustay_id', 'label', 't_0', 'uom', 'value']
ventcategory : ['delta', 'icustay_id', 'label', 't_0', 'uom', 'value']
height : ['delta', 'icustay_id', 'label', 't_0', 'uom', 'value']
norepinephrine : ['delta', 'icustay_id', 'label', 't_0', 'uom', 'value']
weight : ['delta', 'icustay_id', 'label', 't_0', 'uom', 'value']
gcs : ['delta', 'icustay_id', 'label', 't_0', 'uom', 'value']
labs : ['delta', 'icustay_id', 'label', 't_0', 'uom', 'value']
dobutamine : ['delta', 'icustay_id', 'label', 't_0', 'uom', 'value']
vitals : ['charttime', 'delta', 'icustay_id', 'label', 't_0', 'uom', 'value']
phenylephrine : ['delta', 'icustay_id', 'label', 't_0', 'u

In [69]:
#dropping charttime, endtime and first_charttime

list1=[]
list2=[]
for element in df_list:
    if 'charttime' in (list(dfs[element])):
        list1.append(element)
    if 'endtime' in (list(dfs[element])):
        list2.append(element)

for element in list1:
    dfs[element]= dfs[element].drop('charttime', axis=1)

for element in list2:
    dfs[element]= dfs[element].drop('endtime', axis=1)
    
#dfs['rrt']= dfs['rrt'].drop('first_charttime', axis=1)

time: 333 ms


In [70]:
#converting valuenum and value to same label
list1=[]

for element in df_list:
    if 'valuenum' in (list(dfs[element])):
        list1.append(element)

for element in list1:
    dfs[element]= dfs[element].rename(index=str, columns={'valuenum':'value'})


time: 120 ms


In [71]:
del(list1,list2)

time: 592 µs


In [72]:
def label_lower(df_name):
    dfs[df_name]['label']= dfs[df_name]['label'].apply(lambda x: x.lower())

time: 999 µs


In [73]:
#turning all labels to lowercase
for element in df_list:
    label_lower(element)


time: 1.82 s


In [74]:
#adding a df source table label to each df. 
for element in df_list:
    dfs[element]['source']=element

#adding a patient id to each

for element in df_list:
    dfs[element]= pd.merge(dfs[element], final_pt_df2[['icustay_id','subject_id']], how='left')
    

time: 2.64 s


In [75]:
#all of the column names 
for element in df_list:
    print(element,':',sorted(list(dfs[element])))

leuk : ['delta', 'icustay_id', 'label', 'source', 'subject_id', 't_0', 'uom', 'value']
vasopressin : ['delta', 'icustay_id', 'label', 'source', 'subject_id', 't_0', 'uom', 'value']
bg_ART : ['delta', 'icustay_id', 'label', 'source', 'subject_id', 't_0', 'uom', 'value']
rrt : ['delta', 'icustay_id', 'label', 'source', 'subject_id', 't_0', 'uom', 'value']
epinephrine : ['delta', 'icustay_id', 'label', 'source', 'subject_id', 't_0', 'uom', 'value']
pt_info : ['delta', 'icustay_id', 'label', 'source', 'subject_id', 't_0', 'uom', 'value']
ventcategory : ['delta', 'icustay_id', 'label', 'source', 'subject_id', 't_0', 'uom', 'value']
height : ['delta', 'icustay_id', 'label', 'source', 'subject_id', 't_0', 'uom', 'value']
norepinephrine : ['delta', 'icustay_id', 'label', 'source', 'subject_id', 't_0', 'uom', 'value']
weight : ['delta', 'icustay_id', 'label', 'source', 'subject_id', 't_0', 'uom', 'value']
gcs : ['delta', 'icustay_id', 'label', 'source', 'subject_id', 't_0', 'uom', 'value']
labs

In [76]:
pd.to_timedelta(dfs['norepinephrine']['delta']).describe()

count                     30231
mean     0 days 12:56:34.480500
std      0 days 14:25:15.030672
min             0 days 00:00:00
25%             0 days 00:00:00
50%             0 days 08:13:00
75%             0 days 23:15:00
max             2 days 00:00:00
Name: delta, dtype: object

time: 116 ms


# converting data formats

# looking at measured values

In [78]:
df_list

['leuk',
 'vasopressin',
 'bg_ART',
 'rrt',
 'epinephrine',
 'pt_info',
 'ventcategory',
 'height',
 'norepinephrine',
 'weight',
 'gcs',
 'labs',
 'dobutamine',
 'vitals',
 'phenylephrine',
 'dopamine',
 'o2_flow',
 'nit',
 'pafaRatio',
 'cancer_elix',
 'sofa',
 'bg_all',
 'sum_elix']

time: 1.95 ms


In [79]:
def value_viewer(df_name):
    return(dfs[df_name]['label'].unique())

time: 1.25 ms


In [80]:
value_viewer('bg_ART')

array(['pco2'], dtype=object)

time: 3.39 ms


In [81]:
value_viewer('bg_all')

array(['calcium', 'lactate', 'ph', 'temperature', 'hemoglobin',
       'potassium', 'chloride', 'glucose', 'sodium', 'bicarbonate'],
      dtype=object)

time: 9.32 ms


In [82]:
#value_viewer('uti')

time: 485 µs


In [83]:
value_viewer('labs')


array(['sodium', 'bicarbonate', 'chloride', 'creatinine', 'glucose',
       'potassium', 'bun', 'hemoglobin', 'inr', 'platelet', 'ptt', 'wbc',
       'lactate', 'bilirubin', 'bands'], dtype=object)

time: 29.9 ms


In [84]:
value_viewer('vitals')

array(['meanartpress', 'sysbp', 'heartrate', 'resprate', 'spo2',
       'temperature', 'glucose', 'diasbp'], dtype=object)

time: 214 ms


In [112]:
value_viewer('pt_info')

array(['first_admit_age', 'ethnicity', 'gender'], dtype=object)

time: 5.47 ms


In [85]:
list(dfs)

['leuk',
 'vasopressin',
 'bg_ART',
 'rrt',
 'epinephrine',
 'pt_info',
 'ventcategory',
 'height',
 'norepinephrine',
 'weight',
 'gcs',
 'labs',
 'dobutamine',
 'vitals',
 'phenylephrine',
 'dopamine',
 'o2_flow',
 'nit',
 'pafaRatio',
 'cancer_elix',
 'sofa',
 'bg_all',
 'sum_elix']

time: 1.98 ms


# combining data

In [86]:
set(value_viewer('labs')) & set(value_viewer('bg_all'))

{'bicarbonate',
 'chloride',
 'glucose',
 'hemoglobin',
 'lactate',
 'potassium',
 'sodium'}

time: 38 ms


In [87]:
set(value_viewer('labs')) & set(value_viewer('vitals'))

{'glucose'}

time: 240 ms


In [88]:
set(value_viewer('bg_all')) & set(value_viewer('vitals'))

{'glucose', 'temperature'}

time: 203 ms


In [89]:
# (dfs['labs'].loc[
#     dfs['labs'].loc[:,'label']=='glucose', ['label','value']
# ]
#      .groupby(['label'])
#          .describe(percentiles=[.25, .5, .75,.95, .99])
#               )

time: 813 µs


In [90]:
# (dfs['vitals'].loc[
#     dfs['vitals'].loc[:,'label']=='glucose', ['label','value']
# ]
#      .groupby(['label'])
#          .describe(percentiles=[.25, .5, .75,.95, .99])
#               )

time: 899 µs


In [91]:
# (dfs['bg_all'].loc[
#     dfs['bg_all'].loc[:,'label']=='glucose', ['label','valuenum']
# ]
#      .groupby(['label'])
#          .describe(percentiles=[.25, .5, .75,.95, .99])
#               )

time: 1.06 ms


### merging labs together

In [92]:
# lab_bg_vital= pd.concat([dfs['labs'],dfs['bg_all'],dfs['vitals']], sort=False).sort_values(['icustay_id','delta','label','source'], ascending=True)

time: 505 µs


In [93]:
# lab_bg_vital.head()

time: 531 µs


In [94]:
# #rounding timedeltas to the 2 minute mark. 
# #pd.to_datetime(lab_bg_vital['delta'])#.dt.round('m')
# lab_bg_vital['delta']= pd.to_timedelta(lab_bg_vital['delta'])
# lab_bg_vital['delta']= pd.to_datetime(lab_bg_vital['delta']).dt.round('2min')
# lab_bg_vital['delta']= pd.to_timedelta(lab_bg_vital['delta'])


# #note this is more efficient than rounding timedeltas for some reason. 

time: 1.95 ms


In [95]:
# lab_bg_vital.drop_duplicates(subset=['icustay_id','label','value','delta',], keep='last', inplace=False) #n=7001349 at 1 min, 6913527 at 2min rounding,  vs 7222647 without. 

time: 503 µs


In [96]:
# list(lab_bg_vital['label'].unique())

time: 394 µs


# combining all df

In [97]:
#testing combining all df's
##this may not be a useful exercise, but experimenting. 

time: 569 µs


In [98]:
#big_df= pd.concat([dfs['labs'],dfs['bg_all'],dfs['vitals']], sort=False).sort_values(['icustay_id','delta','label','source'], ascending=True)

time: 460 µs


In [99]:
# making one big dataframe via pd. concat
big_df= pd.concat(dfs.values(), sort=False).sort_values(['icustay_id','delta','label','source'], ascending=True)
#converting delta to time delta, to datetime rounded to 2 minutes, and back to time delta (more efficient than rounding timedeltas)
big_df['delta']= pd.to_timedelta(big_df['delta'])
big_df['delta']= pd.to_datetime(big_df['delta']).dt.round('2min')
big_df['delta']= pd.to_timedelta(big_df['delta'])

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2910: FutureWarning: Passing timedelta64-dtype data is deprecated, will raise a TypeError in a future version
  exec(code_obj, self.user_global_ns, self.user_ns)


time: 29.5 s


/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: Passing datetime64-dtype data to TimedeltaIndex is deprecated, will raise a TypeError in a future version
  


In [100]:
len(big_df)

5663255

time: 2.3 ms


In [101]:
big_df= big_df.drop_duplicates(subset=['icustay_id','label','value','delta',], keep='last', inplace=False) #7638425  -> 7315304 at 2 min. 

time: 2.63 s


In [102]:
len(big_df)

5427646

time: 2.13 ms


In [103]:
#big_df['sum_elix']

time: 727 µs


In [104]:
big_df.groupby('label')['value'].describe() #14478 icustay_id's

,count,unique,top,freq
label,,,,
bands,14181,3,absent,11919
bicarbonate,45045,56,23,3720
bilirubin,13459,370,0.4,1163
bun,44760,198,14,1345
calcium,22687,167,1.13,1158
cancer_elix,14181,2,0,12439
chloride,49703,86,106,3183
creatinine,44947,182,0.7,3699
daily_sofa,30669,23,1,5664


time: 1.93 s


In [105]:
big_df['label'].unique()

array(['cancer_elix', 'daily_sofa', 'dobutamine', 'dopamine',
       'epinephrine', 'ethnicity', 'first_admit_age', 'gender',
       'norepinephrine', 'o2_flow', 'pao2fio2ratio', 'pco2',
       'phenylephrine', 'rrt', 'sum_elix', 'vasopressin', 'vent_recieved',
       'weight', 'lactate', 'mingcs', 'diasbp', 'heartrate',
       'meanartpress', 'resprate', 'spo2', 'sysbp', 'temperature',
       'bands', 'hemoglobin', 'platelet', 'wbc', 'leukocyte', 'nitrite',
       'height', 'calcium', 'glucose', 'ph', 'bicarbonate', 'bun',
       'chloride', 'creatinine', 'inr', 'potassium', 'ptt', 'sodium',
       'bilirubin'], dtype=object)

time: 470 ms


In [106]:
big_df[big_df['label']=='leukocyte'].head()

,icustay_id,t_0,delta,value,label,uom,source,subject_id
6198,200012.0,2153-12-23,1 days,Neg/Not_tested,leukocyte,pos/neg category,leuk,28448
1525,200014.0,2105-02-17,1 days,Neg/Not_tested,leukocyte,pos/neg category,leuk,9514
4469,200019.0,2178-07-09,1 days,Neg/Not_tested,leukocyte,pos/neg category,leuk,21789
2820,200030.0,2150-11-14,1 days,Neg/Not_tested,leukocyte,pos/neg category,leuk,14987
9256,200033.0,2198-08-11,1 days,Neg/Not_tested,leukocyte,pos/neg category,leuk,56369


time: 387 ms


In [107]:
big_df['icustay_id'].nunique() #also odd this is not 14668

14181

time: 41.3 ms


## last minute data cleaning

In [108]:
#removing firstpos else neg ssc col
big_df=big_df.loc[:,list(big_df.columns!="first_pos_else_neg_ssc")]

time: 542 ms


In [109]:
wd

'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling'

time: 1.96 ms


In [110]:
date

'16052019'

time: 2.74 ms


In [111]:
pd.DataFrame(big_df).to_csv(Path(
    wd+'/data/processed/merged/{}_longdf_preImp_{}.csv'.format(date,timewindowdays)))

time: 1min 49s


In [56]:
del big_df
del dfs

time: 272 ms


#### making a patient missingness visualization

In [57]:
# #big_agg= big_df.groupby(['icustay_id','label'], as_index=False)['value'].agg(['min'])
# big_agg= big_df.groupby(['icustay_id','label'], as_index=False)['value'].size()
# big_agg_count= big_agg.reset_index().pivot(index='icustay_id',columns='label', values=0)#, levels='icustay_id')

time: 778 µs


In [58]:
# big_agg_count= big_agg.reset_index().pivot(index='icustay_id',columns='label', values=0)#, levels='icustay_id')

time: 473 µs


In [59]:
# big_agg_count

time: 437 µs


In [60]:
# sns.set(rc={'figure.figsize':(25,15)})

time: 434 µs


In [61]:
# #big_agg_min

# #%matplotlib inline
# sns.set(rc={'figure.figsize':(25,15)})
# big_agg_count= big_agg_count.fillna(0)
# big_agg_count = big_agg_count[big_agg_count.columns].astype(float) 


# sns.heatmap(big_agg_count,vmin=0, vmax=1, cmap=sns.color_palette("RdBu_r", 5))

time: 1.1 ms


In [62]:
###f

time: 475 µs


In [63]:
# len(list(big_agg_count)) #38 columns. 

time: 422 µs


In [64]:
# big_agg_count

time: 441 µs


In [65]:
# big_agg_count[big_agg_count>0] =1
# big_agg_pt_missing= big_agg_count.T.apply(lambda x:100*(len(list(big_agg_count))-sum(x))/len(list(big_agg_count)))
# big_agg_pt_missing= pd.DataFrame(big_agg_pt_missing).rename(index=str, columns={0:'%_of_values_missing'})

time: 713 µs


In [66]:
# big_agg_pt_missing.sort_values('%_of_values_missing',ascending=False).plot()

time: 464 µs


In [67]:
# len(big_agg_pt_missing)

time: 646 µs


In [68]:
# big_agg_pt_missing.describe()

time: 596 µs


# 04-08-19: need to visualize current data. need to see matrix of patient by time. 

see 07.X-PatientTime for this

In [69]:
# list1=[]
# list2=[]
# for element in df_list:
#     if 'value' in (list(dfs[element])):
#         list1.append(element)
#     if 'valuenum' in (list(dfs[element])):
#         list2.append(element)




time: 1.16 ms


In [70]:
# for element in list2:
#     print(dfs[element].groupby('label')['valuenum'].describe())

time: 464 µs


In [71]:
# for element in list1:
#     print(dfs[element].groupby('label')['value'].describe())

time: 829 µs


In [72]:
# dfs['bg_ART'].groupby('label')['valuenum'].apply(lambda x: type(x))

time: 833 µs
